In [60]:
import gpxpy
import gpxpy.gpx
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [95]:
# ref https://github.com/marcusvolz/strava_py/blob/main/src/stravavis/process_data.py for processing gpx for strava and plot functions
def process_files(gpx_file):
    with open(f"/Users/jaidenratti/Downloads/export_67811653/activities/{gpx_file}", encoding="utf-8") as f:
        try:
            activity = gpxpy.parse(f)
        except gpxpy.mod_gpx.GPXException as e:
            return None
    
    lat = []
    lon = []
    elev = []
    dist = []
    time = []
    name = []

    for track in activity.tracks:
        for segment in track.segments:
            x0 = activity.tracks[0].segments[0].points[0].longitude
            y0 = activity.tracks[0].segments[0].points[0].latitude
            d0 = 0
            for point in segment.points:
               x = point.longitude
               y = point.latitude
               z = point.elevation
               t = point.time
               lon.append(x)
               lat.append(y)
               elev.append(z)
               time.append(t)
               name.append(gpx_file)
               distance = d0 + math.sqrt(math.pow(x-x0,2) + math.pow(y-y0,2))
               dist.append(distance)
               x0 = x
               y0 = y
               d0 = distance
    
    df = pd.DataFrame(
        list(zip(lon,lat,elev,time,name,dist)),
        columns=["lon","lat","elev","time","name","dist"]
    )
    return df


In [ ]:
gpx_dir = '/Users/jaidenratti/Downloads/export_67811653/activities'
gpx_files = [f for f in os.listdir(gpx_dir) if f.endswith('.gpx')]
gpx_files.sort()
processed_data = []

for gpx_file in gpx_files:
  processed_data.append(process_files(gpx_file))

final_df = pd.concat(processed_data,ignore_index=True)
print(final_df)


In [103]:
def plot_elevation(df,output_file="plot_elevation.png"):
    plt.figure()

    start = (
        df.groupby("name").agg({"time":"min"}).reset_index().sort_values("time")
    )
    col = math.ceil(math.sqrt(len(start)))

    pl = sns.FacetGrid(
        data = df,
        col="name",
        col_wrap = col,
        col_order=start["name"],
        sharex=False,
        sharey=True,
        
    )
    pl = pl.map(plt.plot,"dist","elev",color="blue",linewidth=4)
  

    pl.set(xlabel=None)
    pl.set(ylabel=None)
    pl.set(xticks=[])
    pl.set(yticks=[])
    pl.set(xticklabels=[])
    pl.set(yticklabels=[])
    pl.set(yscale="log")
    pl.set_titles(col_template="",row_template="")
    sns.despine(left=True,bottom=True)
    pl.savefig(output_file)


In [ ]:
plot_elevation(final_df,output_file='plot_elevation.png')

In [102]:
def plot_distance(df,output_file="plot_distance.png"):
    plt.figure()

    start = (
        df.groupby("name").agg({"time":"min"}).reset_index().sort_values("time")
    )
    col = math.ceil(math.sqrt(len(start)))

    pl = sns.FacetGrid(
        data = df,
        col="name",
        col_wrap = col,
        col_order=start["name"],
        sharex=False,
        sharey=False,
        
    )
    pl = pl.map(plt.plot,"lon","lat",color="blue",linewidth=4)
  

    pl.set(xlabel=None)
    pl.set(ylabel=None)
    pl.set(xticks=[])
    pl.set(yticks=[])
    pl.set(xticklabels=[])
    pl.set(yticklabels=[])
    pl.set_titles(col_template="",row_template="")
    sns.despine(left=True,bottom=True)
    pl.savefig(output_file)


In [ ]:
plot_distance(final_df,output_file='plot_distance.png')